In [11]:
import csv
import gzip
import os
import sys

# csv.field_size_limit(sys.maxsize)

clinvar_columns = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'INFO']
clinvar_columns_dict = {col: idx for idx, col in enumerate(clinvar_columns)}

clinvar_info_names = [
    'AF_ESP', 'AF_EXAC', 'AF_TGP', 'ALLELEID', 'CLNDN', 'CLNDNINCL', 'CLNDISDB', 'CLNDISDBINCL',
    'CLNHGVS', 'CLNREVSTAT', 'CLNSIG', 'CLNSIGCONF', 'CLNSIGINCL', 'CLNVC', 'CLNVCSO', 'CLNVI',
    'DBVARID', 'GENEINFO', 'MC', 'ONCDN', 'ONCDNINCL', 'ONCDISDB', 'ONCDISDBINCL', 'ONC', 'ONCINCL',
    'ONCREVSTAT', 'ONCCONF', 'ORIGIN', 'RS', 'SCIDN', 'SCIDNINCL', 'SCIDISDB', 'SCIDISDBINCL',
    'SCIREVSTAT', 'SCI', 'SCIINCL'
]
clinvar_info_dict = {info: idx for idx, info in enumerate(clinvar_info_names)}

vep_names = ('Allele|Consequence|IMPACT|SYMBOL|Gene|Feature_type|Feature|BIOTYPE|EXON|INTRON|'
             'HGVSc|HGVSp|cDNA_position|CDS_position|Protein_position|Amino_acids|Codons|'
             'Existing_variation|DISTANCE|STRAND|FLAGS|SYMBOL_SOURCE|HGNC_ID|CANONICAL').split('|')
vep_names_dict = {name: idx for idx, name in enumerate(vep_names)}

headers = ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'CLNSIG', 'CLNVC',  'GENEINFO', 'MC', 
           'Consequence', 'SYMBOL', 'Gene', 'Feature_type', 'Feature', 'BIOTYPE', 'cDNA_position', 'CANONICAL']


def parse_clinvar_vcf(vcf_file, output_dir='', output_filename=''):
    # Если папка не указана, сохраняем в текущую директорию
    if output_dir == '':
        output_dir = os.getcwd()
    
    # Если имя выходного файла не задано, используем имя входного файла, но с расширением .tsv
    # Через basename получаем имя файла, а через splitext оставляем только название без расширения
    if output_filename == '':
        output_filename = os.path.splitext(os.path.basename(vcf_file))[0] + '.tsv'
    else:
        # Если не указано расширение файла (указано неверно), добавляем .tsv
        if not output_filename.endswith('.tsv'):
            output_filename += '.tsv'
    
    # Полный путь к выходному файлу
    output_file = os.path.join(output_dir, output_filename)
    
    # Запись шапки таблицы в выходной файл
    with open(output_file, 'w', newline='', encoding='utf-8') as table_file:
        writer = csv.writer(table_file, delimiter='\t')
        writer.writerow(headers)

    
    with gzip.open(vcf_file, 'rt') as input_file, open(output_file, 'a', newline='', encoding='utf-8') as output_file:
        vcf_reader = csv.reader(input_file, delimiter='\t')
        tsv_writer = csv.writer(output_file, delimiter='\t')
        
        for line in vcf_reader:
            parsed_data = parse_vcf_line(line)
            for data in parsed_data:
                tsv_writer.writerow(data)


def parse_vcf_line(line):
    parsed_data = []

    if not line[clinvar_columns_dict['CHROM']].startswith('#'):
        chrom = 'chr' + line[clinvar_columns_dict['CHROM']]
        position = line[clinvar_columns_dict['POS']]
        variation_id = line[clinvar_columns_dict['ID']]
        ref = line[clinvar_columns_dict['REF']]
        alt = line[clinvar_columns_dict['ALT']]

        info = line[clinvar_columns_dict['INFO']].split(';')
        for element in info:
            for category in clinvar_info_dict:
                if element.startswith(f'{category}='):
                    value = element.split('=')
                    clinvar_info_dict[category] = value[-1] 
        
        vep_info = info[-1].split(',')
        
        for element in vep_info:
            variant_info = element.split('|')

            info_consequence = variant_info[vep_names_dict.get('Consequence')]
            info_symbol = variant_info[vep_names_dict.get('SYMBOL')]
            info_gene = variant_info[vep_names_dict.get('Gene')]
            info_feature_type = variant_info[vep_names_dict.get('Feature_type')]
            info_feature = variant_info[vep_names_dict.get('Feature')]
            info_biotype = variant_info[vep_names_dict.get('BIOTYPE')]
            info_cDNA = variant_info[vep_names_dict.get('cDNA_position')]
            info_canonical = variant_info[vep_names_dict.get('CANONICAL')]
            
            filtered_data = [
                chrom, position, variation_id, ref, alt,
                clinvar_info_dict['CLNSIG'], clinvar_info_dict['CLNVC'], clinvar_info_dict['GENEINFO'], clinvar_info_dict['MC'],
                info_consequence, info_symbol, info_gene, info_feature_type, 
                info_feature, info_biotype,
                info_cDNA, info_canonical
            ]
            parsed_data.append(filtered_data)

    return parsed_data


Указываем input файл (или путь к нему), а также имя output (необязательно, задаётся по умолчанию по названию вцф):

In [10]:
parse_clinvar_vcf('data_dir/clinvar/clinvar_20240331_anno.vcf.gz', output_filename='clinvar_20240331_anno.vcf.TEST')